## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [1]:
import numpy as np
from numba import njit, jit
import time

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Función para simular %timeit
def my_timeit(func, *args, n=5, r=2):
    all_times = []
    for _ in range(r):
        start = time.perf_counter()
        for _ in range(n):
            result = func(*args)
        end = time.perf_counter()
        all_times.append((end - start) / n)
    
    mean_time = np.mean(all_times) * 1000  # convertir a ms
    std_time = np.std(all_times) * 1000 * 1000  # convertir a μs
    
    # Formatear salida como %timeit
    if mean_time < 1:
        mean_str = f"{mean_time*1000:.2f} μs"
        std_str = f"{std_time:.1f} ns"
    elif mean_time < 10:
        mean_str = f"{mean_time:.2f} ms"
        std_str = f"{std_time:.1f} μs"
    else:
        mean_str = f"{mean_time:.1f} ms"
        std_str = f"{std_time:.1f} μs"
    
    print(f"{mean_str} ± {std_str} per loop (mean ± std. dev. of {r} runs, {n} loops each)")

# Numba Python: huge improvement, better that numpy code
my_timeit(grade2_vector, a_cpu, b_cpu, a, b, c, n=5, r=2)

# w/ a numpy ufunc manually coded
my_timeit(grade2_ufunc, a_cpu, b_cpu, a, b, c, n=5, r=2)

# using the general numpy ufunc 
my_timeit(lambda: a*a_cpu**2 + b*b_cpu + c, n=5, r=2)

float64
8.65 ms ± 29.1 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
19.0 ms ± 25.7 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
19.1 ms ± 40.0 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


## GPU con cupy

In [2]:
import numpy as np
import cupy as cp
from cupyx.profiler import benchmark

def grade2_ufunc(x, y, a, b, c):
    return a*x*x + b*y + c

size = 5_000_000

a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10.0

print(a_cpu.dtype)

def grade2_cupy_with_copy(x_cpu, y_cpu, a, b, c):
    x_gpu = cp.asarray(x_cpu)
    y_gpu = cp.asarray(y_cpu)
    z_gpu = grade2_ufunc(x_gpu, y_gpu, a, b, c)
    return cp.asnumpy(z_gpu)

_ = grade2_cupy_with_copy(a_cpu, b_cpu, a, b, c)

tiempo = benchmark(
    grade2_cupy_with_copy,
    (a_cpu, b_cpu, a, b, c),
    n_repeat=5
)

print("Time with copy:")
print(tiempo.gpu_times.mean(), "±", tiempo.gpu_times.std())

a_gpu = cp.random.rand(size, dtype=cp.float64)
b_gpu = cp.random.rand(size, dtype=cp.float64)

_ = grade2_ufunc(a_gpu, b_gpu, a, b, c)

tiempo = benchmark(
    grade2_ufunc,
    (a_gpu, b_gpu, a, b, c),
    n_repeat=5
)

print("\nTime without copy:")
print(tiempo.gpu_times.mean(), "±", tiempo.gpu_times.std())

float64
Time with copy:
0.018155039978027344 ± 0.0001276826854245235

Time without copy:
0.002018892812728882 ± 3.054137354243286e-06


## GPU con Numpy

In [3]:
import numpy as np
import cupy as cp
import time
from numba import vectorize

@vectorize(
    ['float64(float64, float64, float64, float64, float64)'],
    target='cuda'
)
def grade2_numba_gpu(x, y, a, b, c):
    return a*x*x + b*y + c

size = 5_000_000

a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)

a = 3.5
b = 2.8
c = 10.0

print(a_cpu.dtype)

def compute_with_copy():
    return grade2_numba_gpu(a_cpu, b_cpu, a, b, c)

_ = compute_with_copy()

times = []
for i in range(5):
    t0 = time.time()
    _ = compute_with_copy()
    t1 = time.time()
    times.append(t1 - t0)

print("Time with copy:")
print(np.mean(times), "±", np.std(times))

a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

def compute_no_copy():
    return grade2_numba_gpu(a_gpu, b_gpu, a, b, c)

_ = compute_no_copy()

times = []
for i in range(5):
    t0 = time.time()
    _ = compute_no_copy()
    t1 = time.time()
    times.append(t1 - t0)

print("\nTime without copy:")
print(np.mean(times), "±", np.std(times))

float64
Time with copy:
0.01915316581726074 ± 0.002195424873061129

Time without copy:
0.0025166034698486327 ± 0.0014511907664684178


## Analisis de resultados

En primer lugar, si se toma como referencia la ejecución en CPU, se aprecia que el uso de Numba en modo secuencial supone una mejora clara frente al código original. Mientras que las versiones basadas en ufuncs de NumPy presentan tiempos cercanos a 0.018 segundos, la versión con Numba reduce el tiempo de ejecución hasta aproximadamente 0.008 segundos, lo que indica que la compilación JIT ya resulta efectiva incluso sin recurrir a la GPU.

Por otro lado, al ejecutar el cálculo en GPU mediante la librería CuPy, se observa una diferencia muy marcada entre los casos en los que es necesario copiar los datos desde la CPU y aquellos en los que los datos se generan directamente en la GPU. En el primer caso, cuando existe transferencia de datos entre CPU y GPU, el tiempo total se sitúa en torno a 0.018 segundos, lo que muestra que el coste de dicha copia tiene un impacto notable en el rendimiento global.

Sin embargo, cuando los arrays se crean directamente en la GPU y se evita la transferencia de datos, el tiempo de ejecución se reduce considerablemente hasta aproximadamente 0.0019 segundos. De este modo, queda claro que el cálculo en la GPU es muy rápido y que la principal penalización aparece únicamente cuando se incluyen operaciones de copia de memoria.

De manera similar, al emplear Numba para ejecutar el cálculo en GPU se obtienen resultados coherentes con los observados en CuPy. Cuando la copia de datos entre CPU y GPU se realiza de forma automática, el tiempo vuelve a situarse alrededor de 0.018 segundos, mientras que, al copiar los datos manualmente fuera de la medición y mantenerlos en la GPU, el tiempo desciende hasta aproximadamente 0.0016 segundos, alcanzando rendimientos prácticamente equivalentes a los de CuPy.